In [133]:

            # Get rid of any place where the longitude is equal to 360
            # Not needed by Eliza Code
            df = df.reset_index()

            def get_incident_flux(df, system_obliquity):
                """Add a new column corresponding to the fraction of intensity"""
                
                # Find the incident fraction depending on latitude and longitude
                df['incident_frac'] = np.sin(df.lat * np.pi / 180.0) * np.sin(system_obliquity) + \
                                      np.cos(df.lat * np.pi / 180.0) * np.cos(system_obliquity) * np.cos(df.lon * np.pi / 180.0)

                # Anywhere not on the circle illuminated set to 0
                #df.incident_frac = df.incident_frac.mask(df.incident_frac < 0.0, 0.0)
                return df

            def phase_rotation(df, phase):
                """ Rotate the planet a certain phase and rollover longitude"""
                # Rotate by the given phase
                df.lon = df.lon + phase

                # Rollover the longitude values if they are too big
                df.lon = df.lon.mask(df.lon >= 360.0, df.lon - 360.0)
                return df


            def wind_rot(df):
                """ Calculate the new u wind speed"""
                u = df.u
                v = df.v
                lon = df.lon
                lat = df.lat

                obs_theta_degree = 180.0 - phase
                obs_theta = obs_theta_degree * (np.pi / 180.0)
                obs_theta = 0

                # Convert to radians
                phi = lat * (np.pi / 180.0)
                theta = lon * (np.pi / 180.0)

                # Get the prime cordinates
                phi_prime = np.arcsin(np.cos(inc) * np.sin(phi) -
                                      np.cos(theta) * np.sin(inc) * np.cos(phi))

                theta_prime = np.arctan2(np.sin(theta) * np.cos(phi),
                                         np.cos(inc) * (np.cos(theta) * np.cos(phi)) + np.sin(inc) * np.sin(phi))

                # Update the lat and lon with the new rotation
                lat_prime = phi_prime * (180.0 / np.pi)
                lon_prime = theta_prime * (180.0 / np.pi)

                # I am not including any w terms in the vector rotation
                v_x = u * np.sin(theta - obs_theta) + v * np.cos(theta - obs_theta) * np.sin(phi)
                v_y = -u * np.cos(theta - obs_theta) + (v * np.sin(theta - obs_theta) * np.sin(phi))
                v_z = v * np.cos(phi)

                # All this vector math is in Deryl's google doc
                v_x_prime = -v_x * np.cos(inc) - (v_z * np.sin(inc))
                v_y_prime = -v_y
                v_z_prime = v_x * np.sin(inc) + (v_z * np.cos(inc))

                u_prime = (-v_x_prime) * np.sin(theta_prime) + v_y_prime * np.cos(theta_prime)
                v_prime = np.sin(phi_prime) * ((-v_x_prime * np.cos(theta_prime)) - v_y_prime * np.sin(theta_prime)) + v_z_prime * np.cos(phi_prime)

                # Make everything mod 360
                if lat_prime < -90.0:
                    lat_prime = 180.0 + lat_prime
                elif lat_prime > 90.0:
                    lat_prime = lat_prime - 180.0

                if lon_prime < 0.0:
                    lon_prime = 360.0 + lon_prime
                elif lon_prime >= 360.0:
                    lon_prime = lon_prime - 360.0
                 
                df.lat = lat_prime
                df.lon = lon_prime
                df.u = u_prime
                df.v = v_prime
                return df

            df = get_incident_flux(df, system_obliquity)
            df = phase_rotation(df, phase)
            df = df.apply(wind_rot, axis=1)

-3.202299613496665e-15 -90.0
